# SFTT (Instruction Tuning) using Unsloth

In [ ]:
#install latest version of unsloth
#%pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

#%pip install --upgrade unsloth
# %pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
# %pip install --upgrade unsloth_zoo
# %pip install wandb

In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only

from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset

import torch

# random state
SEED = 42

# reproducibility
## torch
torch.manual_seed(SEED)

## python
import random
random.seed(SEED)

## numpy
import numpy as np
np.random.seed(SEED)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## 🧠 Load Model

In [2]:
max_seq_length = 1024 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_id = "meta-llama/Llama-3.1-8B"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.254 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## Init LoRA

In [4]:
from peft import LoftQConfig
loftq_config = LoftQConfig(loftq_bits=4)

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 256, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 1,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = SEED,
    use_rslora = True,  # We support rank stabilized LoRA
    # init_lora_weights = "loftq",
    # loftq_config = loftq_config, # And LoftQ
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Prepare Data

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

In [2]:
from datasets import DatasetDict, load_dataset
dataset_cs = load_dataset("ctu-aic/cs_instruction_tuning_collection")
dataset_en = load_dataset("ctu-aic/en_instruction_tuning_collection")

README.md:   0%|          | 0.00/4.57k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/152M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/7.46M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/7.76M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/177757 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7614 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7641 [00:00<?, ? examples/s]

In [5]:

dataset_cs["train"].num_rows

103440

In [8]:
train_half = int(len(dataset_cs["train"]) * 0.5)
dev_half = int(len(dataset_cs["validation"]) * 0.5)
test_half = int(len(dataset_cs["test"]) * 0.5)

print(train_half, dev_half, test_half)


51720 1709 1754


In [10]:
SEED = 42
#shuffle the datasets
dataset_cs = dataset_cs.shuffle(seed=SEED)
dataset_en = dataset_en.shuffle(seed=SEED)

#take the first half of the datasets
cs_train = dataset_cs["train"].take(train_half)
en_train = dataset_en["train"].take(train_half)
cs_dev = dataset_cs["validation"].take(dev_half)
en_dev = dataset_en["validation"].take(dev_half)
cs_test = dataset_cs["test"].take(test_half)
en_test = dataset_en["test"].take(test_half)


from datasets import concatenate_datasets
#combine the datasets
dataset = DatasetDict({
    "train": concatenate_datasets([cs_train, en_train]),
    "validation": concatenate_datasets([cs_dev, en_dev]),
    "test": concatenate_datasets([cs_test, en_test]),
})
dataset = dataset.shuffle(seed=SEED)
dataset["train"].num_rows, dataset["validation"].num_rows, dataset["test"].num_rows


(103440, 3418, 3508)

In [11]:
dataset.save_to_disk("/mnt/personal/mlynatom/data/it/mix_11_cs_en")

Saving the dataset (0/1 shards):   0%|          | 0/103440 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3418 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3508 [00:00<?, ? examples/s]

In [5]:
dataset = dataset_cs

In [6]:
from datasets import concatenate_datasets

train = concatenate_datasets([dataset_cs["train"], dataset_en["train"]])
validation = concatenate_datasets([dataset_cs["validation"], dataset_en["validation"]])

In [7]:
from datasets import DatasetDict

train = train.shuffle(seed = SEED)
validation = validation.shuffle(seed = SEED)

train = train.select(range(20000))
validation = validation

dataset = DatasetDict({
    "train" : train,
    "validation" : validation,
})
dataset

DatasetDict({
    train: Dataset({
        features: ['original_id', 'conversations', 'origin', 'instruction_type', 'instruction_translated', 'output_type', 'output_translated'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['original_id', 'conversations', 'origin', 'instruction_type', 'instruction_translated', 'output_type', 'output_translated'],
        num_rows: 11033
    })
})

In [ ]:
# def standardize_ask_library(dataset):
#     """Standardize dataset to hugginface conversations format"""
#     # filter cs language only
#     dataset = dataset.filter(lambda x: x["language"] == "cs")

#     questions = dataset["question"]
#     answers = dataset["answer"]


#     conversations = []
#     for q, a in zip(questions, answers):
#         user = {"role": "user", "content": q}
#         assistant = {"role": "assistant", "content": a}
#         conversations.append([user, assistant])

#     dataset = dataset.add_column("conversations", conversations)

#     return dataset

In [6]:
#dataset = standardize_ask_library(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
dataset = dataset.shuffle(seed = SEED)

In [10]:
dataset["train"][125]["conversations"]

[{'content': 'Vygenerujte si vizualizaci dat, která bude reprezentovat počet absolventů za posledních 5 let.',
  'role': 'user'},
 {'content': 'Jelikož není uvedeno, o jakou oblast absolventů se jedná, budeme pracovat s náhodně vygenerovanými daty. Pro vizualizaci využijeme sloupcový graf.\n\n<img src="https://i.imgur.com/1kAfpwT.png" alt="Graf počtu absolventů za posledních 5 let">\n\nNa grafu je patrné zvýšení počtu absolventů v roce 2019, poté následuje mírný pokles. V roce 2022 se počet absolventů výrazně snížil. Vizualizace nám umožňuje rychle a přehledně porovnat počty absolventů v jednotlivých letech.',
  'role': 'assistant'}]

In [11]:
dataset["train"][100]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nIdentifikujte potenciální problém s internetem a poté navrhněte řešení.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nPotenciálním problémem s internetem může být pomalé nebo nestabilní připojení. Navrhnout lze následující řešení:\n\n1) Zkontrolujte, zda je rychlost vašeho internetového připojení dostatečná pro požadované aktivity. Měřte rychlost pomocí online testů rychlosti internetu a porovnejte ji s rychlostí, kterou vaši poskytovatelé internetu garantují.\n\n2) Připojte se k internetu přes kabel místo Wi-Fi, protože Wi-Fi může být náchylnější k rušení a zpomalování.\n\n3) Zkuste restartovat svůj modem a směrovač (router), abyste obnovili spojení a odstranili případné problémy.\n\n4) Zkontrolujte, zda vás ostatní zařízení, které používají stejné Wi-Fi sítě, nerozptylují šířku pásma. Ujist

### Measurements

In [25]:
#tokenize the texts to see how long the longest sequence is
tokenized = tokenizer(dataset["train"]["text"])

In [27]:
import numpy as np

lengths = [len(ids) for ids in tokenized["input_ids"]]

np.max(lengths), np.mean(lengths), np.std(lengths)

(np.int64(19151), np.float64(363.28375), np.float64(618.5232587671522))

In [28]:
dataset = dataset.shuffle(seed=SEED)

In [12]:
dataset.save_to_disk("data/it/init_mix_cs-en")

Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11033 [00:00<?, ? examples/s]

## Train the model

In [24]:
instruction_template = "<|start_header_id|>user<|end_header_id|>\n\n"
response_template = "<|start_header_id|>assistant<|end_header_id|>\n\n"

data_collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer)

In [25]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = dataset["validation"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    #data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    data_collator = data_collator,
    dataset_num_proc = 8,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 128,
        gradient_accumulation_steps = 8,
        warmup_ratio = 0.01,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 60,
        learning_rate = 8e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = SEED,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        run_name="llama3.1-8B-it-init",
        eval_strategy = "steps",
        eval_steps = 50,
    ),
)

In [15]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=128):   0%|          | 0/103440 [00:00<?, ? examples/s]

Map (num_proc=128):   0%|          | 0/3419 [00:00<?, ? examples/s]

In [26]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nJak se jmenuje hrdina WoD Murph?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHrdina WoD Murph se jmenuje Michael Patrick "Murph" Murphy.<|eot_id|>'

In [28]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["input_ids"]])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nJak se jmenuje hrdina WoD Murph?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHrdina WoD Murph se jmenuje Michael Patrick "Murph" Murphy.<|eot_id|>'

In [24]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nKartavya Path byla nazývána ve jménu<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nzodpovědnosti a povinnosti. Tento termín se obvykle používá v hinduistické filozofii a kultuře a zdůrazňuje důležitost plnění svých povinností a zodpovědností vůči rodině, společnosti a celkově vůči životu. Je to filozofie, která nás vede k tomu, abychom neuplatňovali svá práva, ale abychom se zaměřili na to, co musíme udělat pro ostatní a pro svět jako celek.<|eot_id|>'

In [25]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                              \n\nzodpovědnosti a povinnosti. Tento termín se obvykle používá v hinduistické filozofii a kultuře a zdůrazňuje důležitost plnění svých povinností a zodpovědností vůči rodině, společnosti a celkově vůči životu. Je to filozofie, která nás vede k tomu, abychom neuplatňovali svá práva, ale abychom se zaměřili na to, co musíme udělat pro ostatní a pro svět jako celek.<|eot_id|>'

In [14]:
#Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-80GB. Max memory = 79.254 GB.
18.502 GB of memory reserved.


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 103,440 | Num Epochs = 1 | Total steps = 101
O^O/ \_/ \    Batch size per device = 128 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (128 x 8 x 1) = 1,024
 "-____-"     Trainable parameters = 671,088,640/8,701,349,888 (7.71% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [15]:
#Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

4329.0353 seconds used for training.
72.15 minutes used for training.
Peak reserved memory = 19.932 GB.
Peak reserved memory for training = 17.297 GB.
Peak reserved memory % of max memory = 50.613 %.
Peak reserved memory for training % of max memory = 43.922 %.


## Inference

In [3]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Kdo je nejlepší fotbalista na světě?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 2048, use_cache = True,
                         temperature = 1, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nKdo je nejlepší fotbalista na světě?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTato otázka je vždy aktuální a způsobuje mnoho debat. Nejlepší fotbalista na světě je subjektivní a může se lišit podle osobních preferencí, statistik a individuálních výkonů. Nicméně, podle mnoha odborníků a fanoušků, jsou mezi nejlepšími fotbalisty na světě:\n\n1. **Lionel Messi**: Argentinský útočník, který hraje za Paris Saint-Germain, je často považován za jednoho z nejlepších fotbalistů všech dob. Jeho výjimečné technické dovednosti, rychlost, driblování a střelecké schopnosti ho činí úžasným hráčem.\n2. **Cristiano Ronaldo**: Portugalský útočník, který hraje za Al-Nassr, je další velkým jménem ve fotbalovém světě. Jeho neuvěřitelná síla, rychlost, střelecké schopnosti a vedení na hřišti ho činí jedním z

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nMarie Terezie je<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nDobrý den, Marie Terezie se v českém jazyce nazývá Marie Terézie, je českou královskou královnou. Pozdějí se na dvě části: Marie - z řeckého word Marie (král, prince) a Terezie - z řeckého word Terésia (král) a to v době, kdy se královna pozdívala jako královna a pozdívala se jako královna. Dále se podepisovala jako Marie Terezie, kde se pozdívala jako královna a']